In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
data = pd.read_csv('data/train.csv')
data.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
data = np.array(data)
m, n = data.shape
print(m, n)
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.


42000 785


In [55]:
#test cell
np.random.rand(10, 784) - 0.5

array([[ 0.47642851, -0.41853924, -0.16276699, ..., -0.48637199,
        -0.4963879 ,  0.39171218],
       [-0.44390605,  0.19730862, -0.29999555, ..., -0.29106864,
         0.18733721,  0.31615267],
       [-0.02963343,  0.47986415,  0.15164972, ..., -0.23244536,
         0.32872766, -0.08056538],
       ...,
       [-0.20112119, -0.07665506,  0.3590443 , ...,  0.38356605,
        -0.48456903,  0.08072077],
       [-0.00086359,  0.3241064 ,  0.03381096, ..., -0.27983586,
        -0.22418294,  0.45823883],
       [ 0.42943476,  0.00267741, -0.04965226, ..., -0.09941092,
        -0.49960345,  0.27842383]])

In [66]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def LReLU(Z):
    return np.where(Z > 0, Z, 0.01 * Z)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = LReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * (Z1 > 0)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2



In [67]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [68]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.3, 1001)

Iteration:  0
[1 5 5 ... 4 4 1] [2 8 1 ... 0 1 6]
0.168609756097561
Iteration:  10
[1 9 3 ... 0 4 4] [2 8 1 ... 0 1 6]
0.37170731707317073
Iteration:  20
[1 9 3 ... 0 4 4] [2 8 1 ... 0 1 6]
0.5247560975609756
Iteration:  30
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.637
Iteration:  40
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.6989024390243902
Iteration:  50
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.7346829268292683
Iteration:  60
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.746
Iteration:  70
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.7272439024390244
Iteration:  80
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.7658292682926829
Iteration:  90
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.775390243902439
Iteration:  100
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.788
Iteration:  110
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.798829268292683
Iteration:  120
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.8078780487804879
Iteration:  130
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.8160487804878048
Iteration:  140
[2 8 3 ... 0 1 4] [2 8 1 ... 0 1 6]
0.8227